<a href="https://colab.research.google.com/github/drf0rk/AnxLight/blob/main/notebook/AnxLight_Launcher_v0.0.7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title Cell 1: AnxLight v3 Pre-Flight Setup
# Cell 1: Pre-Flight Setup (v3)
import os
import subprocess
from pathlib import Path
import sys

print("--- AnxLight v3 Pre-Flight Setup Cell ---")

# --- Configuration ---
REPO_URL = "https://github.com/drf0rk/AnxLight.git"
REPO_DIR_NAME = "AnxLight"
BRANCH_NAME = "main"

# --- 1. Clone/Update Repository ---
print(f"--- Ensuring latest repository code from branch: {BRANCH_NAME} ---")
base_dir = Path("/content")
repo_path = base_dir / REPO_DIR_NAME

if not repo_path.exists():
    print(f"Cloning repository '{REPO_URL}' (branch: {BRANCH_NAME}) into '{repo_path}'...")
    subprocess.run(["git", "clone", "-b", BRANCH_NAME, REPO_URL, str(repo_path)], check=True)
else:
    print(f"Repository already exists at '{repo_path}'. Fetching updates from branch '{BRANCH_NAME}'...")
    os.chdir(str(repo_path))
    subprocess.run(["git", "fetch", "origin"], check=True)
    subprocess.run(["git", "checkout", BRANCH_NAME], check=True)
    subprocess.run(["git", "reset", "--hard", f"origin/{BRANCH_NAME}"], check=True)
    subprocess.run(["git", "pull", "origin", BRANCH_NAME], check=True)

os.chdir(str(repo_path))
print(f"Current directory: {os.getcwd()}")

# --- 2. Execute Pre-Flight Setup Script ---
print("\n--- Executing Pre-Flight Setup Script (scripts/pre_flight_setup.py) ---")
pre_flight_script_path = os.path.join(os.getcwd(), "scripts", "pre_flight_setup.py")

if os.path.exists(pre_flight_script_path):
    print(f"Running: {sys.executable} {pre_flight_script_path}")
    # Using Popen to stream output in real-time
    process = subprocess.Popen([sys.executable, pre_flight_script_path],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.STDOUT,
                               text=True,
                               bufsize=1,
                               universal_newlines=True)
    for line in process.stdout:
        print(line, end='')
    process.wait()
    if process.returncode != 0:
        print(f"!!! ERROR: Pre-flight setup script failed with exit code {process.returncode}. Check output above. !!!")
    else:
        print("--- Pre-Flight Setup Script completed successfully. ---")
else:
    print(f"!!! ERROR: scripts/pre_flight_setup.py not found at {pre_flight_script_path}. Cannot proceed. !!!")

print("\n--- AnxLight Cell 1 (Pre-Flight Setup) Complete ---")
print("If no errors above, you can now run Cell 2 to launch the Gradio application.")

--- AnxLight v3 Pre-Flight Setup Cell ---
--- Ensuring latest repository code from branch: main ---
Repository already exists at '/content/AnxLight'. Fetching updates from branch 'main'...
Current directory: /content/AnxLight

--- Executing Pre-Flight Setup Script (scripts/pre_flight_setup.py) ---
Running: /usr/bin/python3 /content/AnxLight/scripts/pre_flight_setup.py
--- AnxLight Pre-Flight Setup ---
AnxLight System Version: 3.0.0-alpha
---------------------------------

--- Pre-Step: Ensuring essential system packages are installed ---
Checking for required packages: ['python3-venv', 'aria2', 'unzip']
Executing: sudo apt-get update -y in .
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/gr

In [ ]:
# @title Cell 2 AnxLight v3 Launch Cell (v0.0.7)
# This cell launches the main Gradio application and monitors for the public URL.

import os
import sys
import subprocess
import time

print("--- AnxLight v3 Launch Cell (v0.0.7) ---")

# --- Get Project Root from Environment (set in Cell 1) ---
project_root = os.environ.get('PROJECT_ROOT', '/content/AnxLight')
print(f"Current Project Root (from env): {project_root}")
if not os.path.isdir(project_root):
    print(f"\\n[CRITICAL ERROR] Project Root '{project_root}' not found!")
    print("Please ensure you have run Cell 1 successfully before running this cell.")
    # Stop execution if the project root isn't valid
    sys.exit(1)

# --- Define Paths ---
venv_python_path = os.path.join(project_root, 'anxlight_venv', 'bin', 'python')
main_gradio_script_path = os.path.join(project_root, 'scripts', 'main_gradio_app.py')

# --- Verify VENV and Script Exist ---
if not os.path.exists(venv_python_path):
    print(f"\\n[CRITICAL ERROR] VENV Python not found at '{venv_python_path}'!")
    print("This indicates Cell 1 did not complete correctly. Please re-run Cell 1.")
    sys.exit(1)

if not os.path.exists(main_gradio_script_path):
    print(f"\\n[CRITICAL ERROR] Main Gradio App not found at '{main_gradio_script_path}'!")
    sys.exit(1)

# --- Set Environment for Gradio Subprocess ---
# These are the same variables from the original notebook, ensuring the app has context
gradio_env = os.environ.copy()
gradio_env['PROJECT_ROOT'] = project_root
gradio_env['home_path'] = project_root
gradio_env['scr_path'] = project_root
gradio_env['settings_path'] = os.path.join(project_root, 'anxlight_config.json')
gradio_env['venv_path'] = os.path.join(project_root, 'anxlight_venv')
# Make modules available to the subprocess
gradio_env['PYTHONPATH'] = f"{os.path.join(project_root, 'modules')}:{project_root}:{os.path.join(project_root, 'scripts')}:{gradio_env.get('PYTHONPATH', '')}"

print("\\n--- Launching Gradio Application Subprocess ---")
print(f"Executing: {venv_python_path} {main_gradio_script_path}")

# --- Launch the subprocess ---
# We will read its output line-by-line in a non-blocking way.
gradio_process = subprocess.Popen(
    [venv_python_path, main_gradio_script_path],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT, # Redirect stderr to stdout to see all messages
    text=True,
    bufsize=1,
    universal_newlines=True,
    env=gradio_env,
    cwd=project_root # Set the working directory for the app
)

# --- Monitor for the Gradio URL without hanging ---
start_time = time.time()
gradio_url_found = False
timeout_seconds = 180 # Wait up to 3 minutes for Gradio to start

print("\\n--- Waiting for Gradio to launch and provide a public URL... ---")
while time.time() - start_time < timeout_seconds:
    # Check if the process has terminated unexpectedly
    if gradio_process.poll() is not None:
        print("\\n[ERROR] Gradio process terminated unexpectedly.")
        # Read any remaining output
        for line in gradio_process.stdout:
            print(line, end='')
        break

    # Read the next line of output
    line = gradio_process.stdout.readline()
    if not line:
        # No output, short sleep to prevent busy-waiting
        time.sleep(0.5)
        continue

    # Print the line to the notebook cell
    print(line, end='')

    # Check for the public URL, which signifies Gradio is ready
    if "Running on public URL:" in line:
        print("\\n" + "="*60)
        print("          GRADIO IS RUNNING AND READY!")
        print("          Click the public URL above to open the UI.")
        print("="*60 + "\\n")
        gradio_url_found = True
        # We found the URL, so we can stop monitoring the log in this loop
        break

# After the loop, check if we timed out
if not gradio_url_found and gradio_process.poll() is None:
    print(f"\\n[TIMEOUT] Gradio did not start and provide a public URL within {timeout_seconds} seconds.")
    print("The process might be hung or taking a very long time to initialize.")
    print("Terminating the process...")
    gradio_process.terminate()

# The cell will now finish, but the gradio_process will continue running in the background
# if the URL was found. If it timed out or crashed, it will be handled above.
print("--- Notebook cell has finished. The Gradio process is running in the background. ---")

# To see continuous logs after this point, you would typically check the log files
# generated by the application itself, if any.

--- AnxLight v3 Launch Cell (v0.0.7) ---
Current Project Root (from env): /content/AnxLight
\n--- Launching Gradio Application Subprocess ---
Executing: /content/AnxLight/anxlight_venv/bin/python /content/AnxLight/scripts/main_gradio_app.py
\n--- Waiting for Gradio to launch and provide a public URL... ---
